# Instalar os pacotes necessários

(Necessário apenas na primeira vez que utilizar tais pacotes)

In [1]:
#!pip install --upgrade pip
#!pip install requests_oauthlib

# Chamar os pacotes que serão utilizados ao longo do código

In [2]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import warnings
from time import gmtime, strftime
from requests_oauthlib import OAuth1Session

warnings.filterwarnings('ignore')

## Autenticação

In [3]:
#API_KEY = "9BGmFKNASlFUv7j7EVwVvUSOr"
#API_SECRET = "Z3TluNxqAmjIQoEoYU8Hh7fmEbWOKKcjYxaizBzYiK9HpZLv3N"
#ACCESS_TOKEN = "2678203501-jMwrB91mYDppQkqUOp3HGsJPT0GGvyPERhQaxpy"
#ACCESS_TOKEN_SECRET = "TUgbMAL53Jd9veKVti6de7f0c1LOmBVtnGEUmHgPPsdVW"

In [4]:
API_KEY = "XIlGfwgJX8ubCE7R5XxayaGN9"
API_SECRET = "BIh5wYEC8EWcBfdrP9qtKJOF4zg04hVOfxxAt4uZ7AVINeJJnt"
ACCESS_TOKEN = "2678203501-jZFsJjk0LO7dywVZRLcEZgC3jzZCn3XogPSpSPa"
ACCESS_TOKEN_SECRET = "MiAwMgECSMBNKhgecFVAxaQcK7wDxJdbroMgxEFzwimpY"

In [5]:
session = OAuth1Session(API_KEY, API_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

## Realizando a busca de fato

### Descobrindo como caracteres especiais devem ser utilizados na busca

In [6]:
TWITTER_URL = 'https://api.twitter.com/1.1/search/tweets.json'
url = TWITTER_URL+'?q=globoplay%20assedio&count=10' # concatenando 2 strings

response = session.get(url)
print(response.status_code)

200


Formas de concatenar STRING

In [7]:
"O nome dele é " + "Leonardo"

'O nome dele é Leonardo'

In [8]:
"Ele tem "+10+" anos"

TypeError: can only concatenate str (not "int") to str

In [9]:
"Ele tem {} anos".format(10)

'Ele tem 10 anos'

E o retorno da API?

In [10]:
response.content

b'{"statuses":[{"created_at":"Thu Oct 18 12:47:17 +0000 2018","id":1052903947734802432,"id_str":"1052903947734802432","text":"RT @guedespri05: T\\u00f4 assistindo a s\\u00e9rie Ass\\u00e9dio da @RedeGlobo @globoplay !!! Adriana Esteves dando um show, o que j\\u00e1 \\u00e9 normal!! Totalmente s\\u2026","truncated":false,"entities":{"hashtags":[],"symbols":[],"user_mentions":[{"screen_name":"guedespri05","name":"Priscila Guedes","id":706481268649824257,"id_str":"706481268649824257","indices":[3,15]},{"screen_name":"RedeGlobo","name":"Globo","id":18219976,"id_str":"18219976","indices":[50,60]},{"screen_name":"globoplay","name":"globoplay","id":55569724,"id_str":"55569724","indices":[61,71]}],"urls":[]},"metadata":{"iso_language_code":"pt","result_type":"recent"},"source":"\\u003ca href=\\"http:\\/\\/twitter.com\\/download\\/android\\" rel=\\"nofollow\\"\\u003eTwitter for Android\\u003c\\/a\\u003e","in_reply_to_status_id":null,"in_reply_to_status_id_str":null,"in_reply_to_user_id":null,"i

In [11]:
print(requests.utils.quote("globoplay assedio"))

globoplay%20assedio


In [12]:
print(requests.utils.quote("#globoplay"))

%23globoplay


### Trabalhando os dados retornados

In [13]:
result_json = json.loads(response.content)
print(result_json.keys())

dict_keys(['statuses', 'search_metadata'])


In [14]:
tweets = result_json['statuses']

In [15]:
print(len(tweets))

10


In [16]:
print(type(tweets[0]))

<class 'dict'>


In [17]:
print(tweets[0].keys())

dict_keys(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang'])


In [18]:
print(tweets[5]['text'])

RT @QuebrandoOTabu: Uma série escrita por uma mulher, dirigida por uma mulher e protagonizada por mulheres. (E com a Adriana Esteves ❤️)
'A…


In [19]:
print(tweets[5]['user'].keys())

dict_keys(['id', 'id_str', 'name', 'screen_name', 'location', 'description', 'url', 'entities', 'protected', 'followers_count', 'friends_count', 'listed_count', 'created_at', 'favourites_count', 'utc_offset', 'time_zone', 'geo_enabled', 'verified', 'statuses_count', 'lang', 'contributors_enabled', 'is_translator', 'is_translation_enabled', 'profile_background_color', 'profile_background_image_url', 'profile_background_image_url_https', 'profile_background_tile', 'profile_image_url', 'profile_image_url_https', 'profile_banner_url', 'profile_link_color', 'profile_sidebar_border_color', 'profile_sidebar_fill_color', 'profile_text_color', 'profile_use_background_image', 'has_extended_profile', 'default_profile', 'default_profile_image', 'following', 'follow_request_sent', 'notifications', 'translator_type'])


In [20]:
print(tweets[1]['user']['name'])

Lauretina 13


## Paginando as buscas

Como se fosse o resultado de uma busca no Google

* Os primeiros 10 resultados são exibidos

* Importante lembrar que o twitter retorna as mensagens ordenadas da mais recente para a mais antiga,
  e os IDs das mensagens são retornados do maior ID para o menor ID.
https://developer.twitter.com/en/docs/tweets/timelines/guides/working-with-timelines.html

* Para ver a próxima página, é necessário informar o menor ID-1 retornado na busca anterior 



In [21]:
result_json['search_metadata']

{'completed_in': 0.053,
 'max_id': 1052903947734802432,
 'max_id_str': '1052903947734802432',
 'next_results': '?max_id=1052766793989849093&q=globoplay%20assedio&count=10&include_entities=1',
 'query': 'globoplay+assedio',
 'refresh_url': '?since_id=1052903947734802432&q=globoplay%20assedio&include_entities=1',
 'count': 10,
 'since_id': 0,
 'since_id_str': '0'}

In [22]:
next_results = result_json['search_metadata']['next_results']
next_results

'?max_id=1052766793989849093&q=globoplay%20assedio&count=10&include_entities=1'

In [23]:
response = session.get(TWITTER_URL+next_results)
print(response.status_code)

200


## Recapitulando ...

In [24]:
# Método para conferir que o menor ID está na última posição e o maior ID está na primeira posição
def get_ids(tweets):
    min_id = 0
    max_id = 0
    for i, tweet in enumerate(tweets):
        if tweet['id'] > max_id:
            max_id = tweet['id']
            print('Maximo na posicao {} => {}'.format(i, max_id))
        if tweet['id'] < max_id:
            min_id = tweet['id']
            print('Minimo na posicao {} => {}'.format(i, min_id))  
    return min_id, max_id

# Método para obter a paginação de forma manual
def get_next_results(tweets, url):
    #min_id, max_id = get_ids(result_json['statuses'])
    max_id = tweets[0]['id']
    min_id = tweets[-1]['id']
    return '?max_id={}&{}'.format(min_id-1, url)

In [ ]:
# Constantes
TWITTER_URL = 'https://api.twitter.com/1.1/search/tweets.json' # STRING
SEARCH_URL = '?q=globoplay&count=100'
N_PAGES = 50

# Variáveis
next_results = SEARCH_URL # STRING
page = 1    # INTEIRO
tweets = [] # LISTA

while next_results is not None and page < N_PAGES: # LOOP
    print("##############")

    response = session.get(TWITTER_URL+next_results)
    if response.status_code != 200:
        print("A página retornou código {}".format(response.status_code))
        break        
    result_json = json.loads(response.content)   

    if not len(result_json['statuses']):
        print("Nenhum tweet retornado")
        break
    tweets_json = result_json['statuses']
    print("Dados carregados - página {} => {} tweets".format(page, len(tweets_json)))

    # Paginação usando o resultado do JSON
    if 'next_results' in result_json['search_metadata']:
        next_results = result_json['search_metadata']['next_results']
    else:
        #next_results = None
        next_results = get_next_results(tweets_json, SEARCH_URL)
    print(next_results)

    page = page+1
    
    # Carregando os resultados
    tweets.extend(tweets_json)

##############
Dados carregados - página 1 => 100 tweets
?max_id=1052895295032516607&q=globoplay&count=100&include_entities=1
##############
Dados carregados - página 2 => 100 tweets
?max_id=1052860591675568129&q=globoplay&count=100&include_entities=1
##############
Dados carregados - página 3 => 100 tweets
?max_id=1052766237996941311&q=globoplay&count=100&include_entities=1
##############
Dados carregados - página 4 => 100 tweets
?max_id=1052739006834900991&q=globoplay&count=100&include_entities=1
##############
Dados carregados - página 5 => 100 tweets
?max_id=1052727306576375807&q=globoplay&count=100&include_entities=1
##############
Dados carregados - página 6 => 100 tweets
?max_id=1052717754665619457&q=globoplay&count=100&include_entities=1
##############


## Transformando a lista de Tweets em uma estrutura do pandas (Dataframe)

In [26]:
df = pd.DataFrame(tweets)
df

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,metadata,place,possibly_sensitive,retweet_count,retweeted,retweeted_status,source,text,truncated,user
0,None,None,Thu Oct 18 12:47:17 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052903947734802432,1052903947734802432,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,2,False,{'created_at': 'Wed Oct 17 13:14:09 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @guedespri05: Tô assistindo a série Assédio...,False,"{'id': 290171850, 'id_str': '290171850', 'name..."
1,None,None,Thu Oct 18 12:45:24 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052903472843120640,1052903472843120640,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 290171850, 'id_str': '290171850', 'name..."
2,None,None,Thu Oct 18 12:12:54 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052895295032516608,1052895295032516608,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 2960202250, 'id_str': '2960202250', 'na..."
3,None,None,Thu Oct 18 11:51:46 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052889973702217729,1052889973702217729,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 3202839129, 'id_str': '3202839129', 'na..."
4,None,None,Thu Oct 18 10:28:36 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052869043596738562,1052869043596738562,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 1020021325069541377, 'id_str': '1020021..."
5,None,None,Thu Oct 18 10:25:51 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052868354342445056,1052868354342445056,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/iphone"" r...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 862854632, 'id_str': '862854632', 'name..."
6,None,None,Thu Oct 18 09:15:54 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052850751989526529,1052850751989526529,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 888470455413342209, 'id_str': '88847045..."
7,None,None,Thu Oct 18 09:09:00 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052849014155165697,1052849014155165697,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a href=""http://twitter.com/download/android"" ...",RT @QuebrandoOTabu: Uma série escrita por uma ...,False,"{'id': 4806129508, 'id_str': '4806129508', 'na..."
8,None,None,Thu Oct 18 03:54:23 +0000 2018,"{'hashtags': [], 'symbols': [], 'user_mentions...",NaN,0,False,None,1052769837339107329,1052769837339107329,...,"{'iso_language_code': 'pt', 'result_type': 're...",None,NaN,148,False,{'created_at': 'Wed Oct 17 22:30:00 +0000 2018...,"<a 

### Salvando os dados em CSV para backup ou para trabalhar em excel, R, etc.

In [27]:
df.to_csv('Dados/tweets_globoplay_{}.csv'.format(strftime("%Y%m%d_%H%M%S", gmtime())), index=False)

In [28]:
df2 = pd.read_csv('Dados/tweets_globoplay_20181017_184453_qGloboplay_3000.csv')
df2.head(10)

FileNotFoundError: File b'tweets_globoplay_20181017_184453_qGloboplay_3000.csv' does not exist

## Analisando o resultado

In [ ]:
print('Quantidade de tweets únicos: {}'.format(len(df.id.unique())))
df.describe()

### Selecionando as colunas desejadas.

In [ ]:
df_created = df[['created_at', 'text']]
df_created.describe()

### Verificando os tipos das colunas:

In [ ]:
df_created.dtypes

### Convertendo a coluna de data (STRING) para uma coluna do tipo date

In [ ]:
df_created['created_at'] = pd.to_datetime(df_created['created_at'])
df_created.dtypes

In [ ]:
pd.set_option('display.max_colwidth', -1)
df_created.head(10)

### Visualizando os dados por dia.

Lembrando que esta API traz os resultados dos últimos 7 dias.

In [ ]:
df_created['day'] = df_created['created_at'].dt.day
df_created.head()

In [ ]:
df_created.hist()

### Visualizando os dados dos dias 15 e 16 por horário

In [ ]:
df_created_15 = df_created[df_created['created_at'].dt.day == 15]
df_created_15['hour'] = df_created_15['created_at'].dt.hour
df_created_15.hist()

In [ ]:
df_created_16 = df_created[df_created['created_at'].dt.day == 16]
df_created_16['hour'] = df_created_16['created_at'].dt.hour
df_created_16.hist()